# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cityweather_csv = "../WeatherPy/Weather_Data.csv"
cityweather_df = pd.read_csv(cityweather_csv)
cityweather_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jamestown,42.0970,-79.2353,67.71,92,0,4.03,US,1656032202
1,aguas formosas,-17.0822,-40.9358,64.36,94,6,1.10,BR,1656032203
2,mogadishu,2.0371,45.3438,77.36,78,93,17.31,SO,1656032205
3,ushuaia,-54.8000,-68.3000,38.86,81,75,16.11,AR,1656032206
4,rikitea,-23.1203,-134.9692,75.78,78,5,17.74,PF,1656032207


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key = g_key)
humidity = cityweather_df["Humidity"].astype(float)
locations = cityweather_df[["Lat", "Long"]].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                 max_intensity= max(cityweather_df["Humidity"]), point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# I'd never go on a winter holiday, because gear and accomodation would cost too much, if going comfortable enough.
# So let's find cold cities, to go on an imaginery holiday.
my_weather_df = cityweather_df
my_weather_df = my_weather_df.loc[(my_weather_df["Max Temp"] > 35) & (my_weather_df["Max Temp"] < 60)]
my_weather_df = my_weather_df.loc[(my_weather_df["Wind Speed"] < 25)]
my_weather_df = my_weather_df.dropna()
my_weather_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,ushuaia,-54.8000,-68.3000,38.86,81,75,16.11,AR,1656032206
9,saskylakh,71.9167,114.0833,53.40,45,99,11.65,RU,1656032212
10,mount gambier,-37.8333,140.7667,54.23,68,100,6.98,AU,1656032213
11,bredasdorp,-34.5322,20.0403,52.20,78,18,5.32,ZA,1656032214
12,cape town,-33.9258,18.4232,57.38,68,20,14.97,ZA,1656032110


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = my_weather_df

# Set params. Hotel didn't work, so changed to accomodationm, restaurant, resort.
params = {"radius":5000, "type":"Restaurant", "key":g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url,params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (IndexError):
        pass

hotel_df = hotel_df.dropna()

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]
markers = gmaps.marker_layer(locations)

In [19]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color= "rgba(0,30, 4, 0)", 
                                 stroke_color= "rgba(0, 0, 140, 0)",
                                 scale= 2, info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))